In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as tfhub


In [2]:
training_set='F:/Birds/train'
test_set='F:/Birds/test'
validation_set= 'F:/Birds/valid'
resnet152= 'https://tfhub.dev/google/imagenet/resnet_v2_152/feature_vector/5'
inceptionv3 = 'https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5'

img_size = 224

In [3]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip( mode='horizontal' ),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.4),
   # tf.keras.layers.experimental.preprocessing.RandomWidth(0.3),
   # tf.keras.layers.experimental.preprocessing.RandomHeight(0.3),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.3)

])
data_aug_test = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)
])

In [4]:
train = tf.keras.preprocessing.image_dataset_from_directory(training_set, labels='inferred', label_mode='int',color_mode='rgb',batch_size=64, image_size=(img_size , img_size),shuffle=True)
num_classes = len(train.class_names)
train= train.map(lambda x,y:(data_aug(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 39364 files belonging to 275 classes.


In [5]:
validation = tf.keras.preprocessing.image_dataset_from_directory(validation_set,label_mode='int',color_mode='rgb',batch_size=64,image_size=(img_size,img_size),shuffle=True,seed=52)
validation = validation.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 1375 files belonging to 275 classes.


In [6]:
test = tf.keras.preprocessing.image_dataset_from_directory(test_set,label_mode='int',color_mode='rgb',batch_size=128,image_size=(img_size,img_size),shuffle=True,seed=52)
test = test.map(lambda x,y:(data_aug_test(x,training=True),y),num_parallel_calls=12).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 1375 files belonging to 275 classes.


In [7]:
resnet = tfhub.KerasLayer(resnet152,trainable=False,input_shape=[img_size,img_size,3])
inception = tfhub.KerasLayer(inceptionv3,trainable=True,input_shape=[img_size,img_size,3])

In [8]:
model = tf.keras.Sequential([
    inception,
    tf.keras.layers.Dense(num_classes,activation='softmax')
])

In [9]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 275)               563475    
Total params: 22,366,259
Trainable params: 22,331,827
Non-trainable params: 34,432
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [11]:
hisotory = model.fit(train, epochs=10, validation_data=validation)

Epoch 1/10
616/616 [==============================] - 117s 176ms/step - loss: 2.0975 - acc: 0.5841 - val_loss: 1.3867 - val_acc: 0.7222
Epoch 2/10
616/616 [==============================] - 102s 166ms/step - loss: 1.0775 - acc: 0.8162 - val_loss: 1.5002 - val_acc: 0.7345
Epoch 3/10
616/616 [==============================] - 102s 166ms/step - loss: 0.8775 - acc: 0.8769 - val_loss: 1.5131 - val_acc: 0.7433
Epoch 4/10
616/616 [==============================] - 106s 173ms/step - loss: 0.8253 - acc: 0.8975 - val_loss: 1.1607 - val_acc: 0.8182
Epoch 5/10
616/616 [==============================] - 113s 183ms/step - loss: 0.7941 - acc: 0.9149 - val_loss: 1.6324 - val_acc: 0.7425
Epoch 6/10
616/616 [==============================] - 112s 182ms/step - loss: 0.7796 - acc: 0.9269 - val_loss: 1.3484 - val_acc: 0.8145
Epoch 7/10
616/616 [==============================] - 112s 182ms/step - loss: 0.7755 - acc: 0.9341 - val_loss: 2.0266 - val_acc: 0.6836
Epoch 8/10
616/616 [============================

In [13]:
model.evaluate(test)

11/11 [==============================] - 1s 76ms/step - loss: 1.1457 - acc: 0.8589


[1.1456815004348755, 0.8589090704917908]